In [10]:
import pandas as pd
tract_join=pd.read_csv('../Datasets/ALL_watersystems_tracts_intersection.txt',encoding = "ISO-8859-1")
tract_info=tract_join[['PWSID','D_POPULATI','GEOID10','ALAND10','AWATER10','AREA_TRACT','AREA_Intersection','Proportion_Int_Tract']]
caldata=pd.read_excel('../Datasets/calenviroscreen.xlsx','CES 3.0 (2018 Update)')

In [23]:
#Read in the CWS data
cws=pd.read_csv('../Datasets/CWS.csv').dropna(subset=['Water System No.'])
def mk_pwsid(row):
    return('CA{:07d}'.format(int(row['Water System No.'])))
cws.loc[:,'PWSID']=cws.apply(mk_pwsid,axis=1)
cws_tract=pd.merge(cws,tract_info)

In [32]:
calsub=caldata[['Census Tract','Total Population','CES 3.0 Score']]
cal_tract=pd.merge(calsub,cws_tract,left_on='Census Tract',right_on='GEOID10')
cal_tract.loc[:,'Tract_Frac_Pop']=cal_tract['Proportion_Int_Tract']*cal_tract['Total Population']

In [36]:
cal_tract.columns

Index(['Census Tract', 'Total Population', 'CES 3.0 Score',
       'Regulating Agency', 'Water System No.', 'Water System Name',
       'Fed Type', 'State Type', 'DS Class', 'Max TP Class', 'Fee Code',
       'Service Connections', 'Population', 'Last SNSV', 'PWSID', 'D_POPULATI',
       'GEOID10', 'ALAND10', 'AWATER10', 'AREA_TRACT', 'AREA_Intersection',
       'Proportion_Int_Tract', 'Tract_Frac_Pop'],
      dtype='object')

In [42]:
cal_merge=cal_tract[['PWSID','Water System Name','Regulating Agency','Fee Code','Service Connections',
                     'Population','Census Tract','Total Population','Proportion_Int_Tract','Tract_Frac_Pop','CES 3.0 Score']]

In [43]:
cal_merge.to_excel('../Datasets/System_Tract_Merge_Info.xlsx',index=False)

In [38]:
#Population weighted values?
sys_stats=pd.DataFrame()
field_calcs=['CES 3.0 Score']
for p in cal_merge['PWSID'].unique():
    csub=cal_merge[cal_merge['PWSID']==p]
    cname=csub['Water System Name'].values[0]
    sys_stats.loc[p,'NAME']=cname
    sys_stats.loc[p,'N']=len(csub)
    sys_stats.loc[p,'Population']=csub['Population'].mean()
    if (len(csub)<2):

        sys_stats.loc[p,'Population']=csub['Population'].values[0]
        for c in field_calcs:
            sys_stats.loc[p,c]=csub[c].values[0]
    else:

        pop_tracts=csub['Tract_Frac_Pop'].sum()
        for c in field_calcs:
            #print(csub[['Tract_Frac_Pop',c]])
            #print((csub['Tract_Frac_Pop']*csub[c]).sum()/csub['Tract_Frac_Pop'].sum())
            sys_stats.loc[p,c]=(csub['Tract_Frac_Pop']*csub[c]).sum()/csub['Tract_Frac_Pop'].sum()
        

In [39]:
sys_stats

,NAME,N,Population,CES 3.0 Score
CA1010007,CITY OF FRESNO,137.0,541990.0,46.583557
CA3610034,ONTARIO MUNICIPAL UTILITIES COMPANY,52.0,181107.0,50.591674
CA3610029,MONTE VISTA CWD,29.0,57678.0,47.757210
CA3910001,CALIFORNIA WATER SERVICE - STOCKTON,51.0,172105.0,55.321439
CA2900530,ROUGH & READY MOBILEHOME VILLAGE,4.0,200.0,72.924703
...,...,...,...,...
CA2701731,BRANDON CREEK MWS,1.0,41.0,NaN
CA2702325,FERNWOOD WS,1.0,133.0,NaN
CA2701278,RANCHO CHAPARRAL MWC,1.0,48.0,NaN
CA2701280,NEPENTHE WS,1.0,50.0,NaN


In [50]:
sys_stats.to_excel('example_pop_weighted_characteristics.xlsx')